# Práctica de reconstrucción.  Parte I. Calibración de cámaras

Visión Computacional 2018-19 <br>
Practica 2. 29 de octubre de 2018

Este enunciado está en el archivo "PracticaCalib2018.ipynb" o su versión "pdf" que puedes encontrar en el Aula Virtual.

## Objetivos


Los objetivos de esta práctica son:
* Calibrar una cámara usando el método de calibración de Zhang, que está implementado en OpenCV.
* Hacer uso de los resultados de la calibración en un sistema simple de realidad aumentada que proyecte un modelo 3D sintético sobre imágenes reales. Esta parte es opcional.
* Calibrar un par de cámaras y deducir información sobre la posición relativa de las mismas.

## Requerimientos

Para esta práctica es necesario disponer del siguiente software:
* Python 2.7 ó 3.X 
* Jupyter http://jupyter.org/.
* Las librerías científicas de Python: NumPy, SciPy, y Matplotlib.
* La librería OpenCV.

El material necesario para la práctica se puede descargar del Aula Virtual en la carpeta ``MaterialesPractica``. Esta
carpeta contiene:
* El enunciado de esta práctica.
* Dos secuencias de imágenes tomadas con un par de cámaras (izquierda y derecha) en los directorios ``left`` y ``right``.
* Tres modelos tridimensionales: la tetera de Utah (teapot), el conejo de Stanford (bunny) y un cubo (cube).   
  La carga de un modelo en Python se realiza como si fuera un módulo. Por ejemplo: ``from models import bunny``.
El módulo cargado contiene dos variables:
  - ``bunny.vertices`` es una matriz $4 × N_v$ con las coordenadas homogéneas de los $N_v$ vértices del modelo (en este caso, el conejo de Stanford). Cada columna son las coordenadas de un vértice.
  - bunny.edges es una matriz 2×N e con los N e arcos del modelo. Cada columna contiene los índices de los dos vértices que une un arco.

## Condiciones

* La fecha límite de entrega será el martes 20 de noviembre a las 23:55.
* La entrega consiste en dos archivos con el código, resultados y respuestas a los ejercicios:
  1. Un "notebook" de Jupyter con los resultados. Las respuestas a los ejercicios debes introducirlas en tantas celdas de código o texto como creas necesarias, insertadas inmediatamente después de  un enuciado y antes del siguiente.
  2. Un documento "pdf" generado a partir del fuente de Jupyter, por ejemplo usando el comando ``jupyter nbconvert --execute --to pdf notebook.ipynb``, o simplemente imprimiendo el "notebook" desde el navegador en la opción del menú "File->Print preview". Asegúrate de que el documento "pdf" contiene todos los resultados correctamente ejecutados.
* Esta práctica puede realizarse en parejas.

## 1. Calibración de una cámara

En esta parte se trabajará con la secuencia de imágenes del directorio ``left``. Esta secuencia contiene una serie de imágenes de la plantilla de calibración. Para la calibración se debe tener en cuenta que el tamaño de cada escaque de la plantilla es de 30 mm en las direcciones X e Y.

In [ ]:
# uncomment to show results in a window
# %matplotlib tk
import cv2
import glob
import copy
import numpy as np
import scipy.misc as scpm
import matplotlib.pyplot as ppl

Implementa la función ``load_images(filenames)`` que reciba una lista de nombres de archivos de imagen
y las cargue como matrices de NumPy. Usa la función ``scipy.misc.imread`` para cargar las imágenes. La función
debe devolver una lista de matrices de NumPy con las imágenes leídas.

In [ ]:
def load_images(filenames):
    """Load multiple images."""
    return ...TODO...

Usa ``load_images`` para cargar todas las imágenes del directorio ``left`` por orden alfabético (la función ``glob.glob`` permite generar la lista de nombres de archivo, y, por ejemplo, la función ``sorted()`` de Python ordena alfabéticamente una lista de cadenas de texto).

La función ``cv2.findChessboardCorners`` de
OpenCV busca la plantilla de calibración en una imagen y
devuelve una tupla de dos elementos. El primer elemento
es 0 si no consiguió detectar correctamente la plantilla, y
es 1 en caso contrario. El segundo elemento contiene las
coordenadas de las esquinas de la plantilla de calibración,
que sólo son válidas si la detección fue exitosa, es decir, si
el primer elemento de la tupla es 1.

**Ejercicio 1.** Usa la  función ``cv2.findChessboardCorners``, y opcionalmente ``cv2.cornerSubPix``, para detectar automáticamente el patrón de calibración y sus esquinas en todas las imágenes cargadas. El tamaño de la plantilla de calibración en las imágenes
de la práctica es (8, 6) , esto es, 8 filas y 6 columnas. Almacena los resultados de las múltiples llamadas en
una lista, de modo que el elemento i de dicha lista corresponda al resultado de ``cv2.findChessboardCorners``
para la imagen i cargada anteriormente.


In [ ]:
corners = ... TODO ... 

In [ ]:
# This section is OPTIONAL
# cornerSubPix is destructive. so we copy standard corners and use the new list to refine
corners2 = copy.deepcopy(corners)

# Refine corner estimation (images mus be in b&w, use cv2.cvtColor(img,cv2.COLOR_RGB2GRAY) to convert from rgb)
# termination criteria (see, e.g https://docs.opencv.org/3.1.0/dc/dbb/tutorial_py_calibration.html)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
cornersRefined = ... TODO ...

El siguiente ejercicio consiste en dibujar sobre las imágenes los puntos detectados por ``cv.FindChessboardCorners``. Por motivos de eficiencia, la función empleada para hacerlo modifica directamente las imagen pasadas por parámetro en lugar de hacer una copia. Para evitar perder las imágenes originales es mejor realizar una copia de las mismas con antelación. Una forma de hacerlo es ``imgs2 = copy.deepcopy(imgs)``
donde ``imgs`` es la lista de imágenes cargadas. Utiliza estas imágenes copiadas en lugar de las
originales en el siguiente ejercicio.

**Ejercicio 2.** Usa ``cv2.drawChessboardCorners`` para dibujar las esquinas detectadas en el ejercicio anterior. Aplícalo a todas las imágenes que fueron correctamente detectadas. Ignora el resto.
Muestra alguna de las imágenes resultantes.

In [ ]:
imgs2 = copy.deepcopy(imgs)

... TODO ...

In [ ]:
ppl.imshow(imgs2[0])

In [ ]:
ppl.imshow(imgs2[1])

Para calibrar la cámara, además de las coordenadas de
las esquinas en cada una de las imágenes, se necesitan las
coordenadas tridimensionales de las esquinas en el sistema
de referencia de la escena. Para esta práctica consideraremos que el centro del sistema de referencia, esto es, el
punto de coordenadas $[0, 0, 0]^\top$, es la primera esquina de
la plantilla de calibración detectada en todas las imágenes.
También consideraremos que el eje X corresponde al lado
corto de la plantilla de calibración, y el eje Y al lado largo.
Esta disposición implica que el eje Z apunta en la dirección
normal hacia arriba del plano de calibración.

Para el siguiente ejercicio es muy importante tener en
cuenta que las coordenadas de las esquinas en el sistema de
referencia de la escena deben darse en el mismo orden que
en el que fueron detectadas en cada una de las imágenes.

**Ejercicio 3.** Implementa la función ``get_chessboard_points(chessboard_shape, dx, dy)`` que genere una matriz de NumPy (es decir, un ndarray) de
tamaño $N × 3$ con las coordenadas $(x,y,z)$ de las esquinas de
la plantilla de calibración en el sistema de referencia de la
escena. $N$ es el número de esquinas de la plantilla.

``chessboard_shape`` es el número de puntos por filas
y por columnas de la plantilla de calibración. Al igual que
en el Ejercicio 1, debe ser (8, 6). ``dx`` (resp. ``dy``) es el ancho
(resp. alto) de un escaque de la plantilla de calibración.
Para la plantilla utilizada en esta práctica, ambos valores
son 30mm.

In [ ]:
def get_chessboard_points(chessboard_shape, dx, dy):    
    return ... TODO ...

cb_points = get_chessboard_points(...)
print(cb_points)

**Ejercicio 4.** Calibra la cámara izquierda usando la lista de resultados de ``cv2.findChessboardCorners``
y el conjunto de puntos del modelo dados por ``get_chessboard_points``, del ejercicio anterior.

Para ello usa la función calibrate que se distribuye con el material de la práctica.
Guarda el resultado de la calibración, matriz de intrínsecos y matrices de extrínsecos, con el comando
np.savez(‘calib_left’, intrinsic=intrinsic, extrinsic=extrinsic)

In [ ]:
# Extract the list of valid images with all corners
valid_corners = ... TODO ...
num_valid_images = len(valid_corners)

# Prepare input data 
# object_points: numpy array with dimensions (number_of_images, number_of_points, 3)
object_points = ... TODO ...
# image_points: numpy array with dimensions (number_of_images, number_of_points, 2)
image_points = ... TODO ...

# Calibrate for square pixels corners standard
rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(... flags=cv2.CALIB_FIX_ASPECT_RATIO)

print("Corners standard intrinsics:\n",intrinsics)
print("Corners standerd dist_coefs:", dist_coeffs)
print("rms:", rms)

###  1.1 Parámetros intrínsecos

Una de las características intrínsecas de una cámara más fácilmente comprensible es su ángulo de visión o campo
de visión (FOV), o el campo de visión de cualquier región en ella. El campo de visión es la amplitud angular de una
determinada escena y se suele expresar en grados. 

**Ejercicio 5.** Conociendo los intrínsecos K y que la región tiene forma rectangular, su esquina superior izquierda está en la posición (10,10) y tiene un tamaño de (50,50) píxeles, calcula el ángulo de visión diagonal que abarca dicha región. Justifica esta solución.

## 2. Realidad aumentada (opcional)

El término *realidad aumentada* hace referencia al conjunto de técnicas que permiten representar información sintética no existente en el mundo real sobre imágenes reales. En nuestro caso, la información
sintética son modelos tridimensionales. Los siguientes ejercicios proponen una serie de pasos para implementar un
pequeño sistema de realidad aumentada, para lo cual serán necesarios los parámetros obtenidos durante la calibración.

**Ejercicio 6.** Implementa la función``m = proj(K, T, verts)`` que, dada la matriz de intrínsecos K (dimensión 3x3), extrínsecos T (dimensión 3 x 4) y una matriz de vértices expresados en coordenadas homogéneas ``verts``, calcule la proyección de los vértices 3D a puntos 2D de la imagen. Las coordenadas 2D resultantes deben ser homogéneas.
Es decir, este ejercicio consiste en implementar la ecuación de proyección vista en clase.

In [ ]:
def proj(K, T, verts):
    return  ... TODO ...

**Ejercicio 7.** Implementa una función ``plothom(points)`` que dibuje un conjunto de puntos 2D de entrada expresados en coordenadas homogéneas.

In [ ]:
def plothom(points):
    ... TODO ...
    ppl.plot(points[0], points[1], '.')

**Ejercicio 8.** Usa las funciones implementadas en los ejercicios anteriores para proyectar un modelo sobre las imágenes de la secuencia. Para ello, modifica la función ``play_ar``, que se distribuye con la práctica, completando las partes marcadas con TODO:

1. Proyecta los vértices del modelo con ``proj`` usando los intrínsecos y los extrínsecos de la imagen que corresponda.
2. Dibuja los vértices proyectados o los arcos correspondientes con ``plothom``.

Prueba la función ``play_ar`` una vez terminada.

In [ ]:
def play_ar(intrinsics, rvecs, tvecs, imgs, vertices):
    
    fig = ppl.gcf()
    
    for rv,tv,img in zip(rvecs, tvecs, imgs):
        fig.clf()
        # ppl.figure()
        
        # Create rotation matrix from rotation vector rv, use cv2.Rodrigues()
        rm = ... TODO ...
        # Create 3 x 4 extrinsics
        T = ... TODO ...
        # Project the model with proj.
        v2d = ... TODO ...

        # TODO: Draw the model with plothom
        
        
        # Plot the image.
        ppl.imshow(img)
        # ppl.draw()
        ppl.show()
        ppl.pause(0.3)

In [ ]:
# read bunny model
from models import bunny

In [ ]:
# play only images with detected corners
valid_imgs = ... TODO ...

play_ar(intrinsics, rvecs, tvecs, valid_imgs, bunny.vertices)

**Ejercicio 9.** Transforma el modelo anterior para que se represente en
el centro de la plantilla de calibración y rotado 90 grados sobre el eje vertical del modelo. 
Ejecuta la función ``play_ar``con el nuevo modelo.

In [ ]:
new_vertices = ... TODO ...

play_ar(intrinsics, rvecs, tvecs, valid_imgs, new_vertices)

## 3. Par de cámaras

**Ejercicio 10.** Siguiendo el procedimiento de la primera parte
de la práctica, calibra la cámara derecha usando la secuencia
de imágenes del directorio ``right``.

In [ ]:
... TODO ..

print("Corners standard intrinsics:\n",intrinsics_r)
print("Corners standerd dist_coefs:", dist_coeffs_r)
print("rms standard:", rms_r)

**Ejercicio 11.** ¿Cuál es la distancia, en milímetros, entre las dos cámaras?

Sugerencia: Utiliza los extrínsecos del primer par de imágenes el que simultáneamente se vean todos los puntos de la plantilla.
